In [ ]:
# Numpy, Scipy, Matplotlib
import scipy.interpolate
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
import numpy as np

# Drake imports
import pydrake
from pydrake.all import (
    DirectCollocation, DirectTranscription, MathematicalProgram,
    InputPortSelection, LogVectorOutput
)
from pydrake.all import FindResourceOrThrow
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator, RigidTransform,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, RollPitchYaw, AutoDiffXd, BodyIndex,
                        RotationMatrix, Meshcat,MeshcatVisualizerParams, MeshcatVisualizerCpp, MeshcatVisualizer,
                        Adder, Gain, ConstantVectorSource, Demultiplexer, Multiplexer, AngleAxis)

# Other imports
import importlib
import re
import enum
from IPython.display import display, SVG, Image
import pydot

# Imports of other project files
import constants
import config
from config import hinge_rotation_axis

import plant.simulation
import plant.manipulator as manipulator


import ctrl.aux
import time

In [ ]:
# Matplotlib configuring
# USE FOR PAPER
# plt.style.use(['science', 'no-latex'])
# font = {'size'   : 14}
# matplotlib.rc('font', **font)
# default_figsize = (2*3,2*2)
# USE GENERALLY
font = {'size'   : 16}
matplotlib.rc('font', **font)
default_figsize = (16,8)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16,8)
matplotlib.rcParams['lines.linewidth'] = 5
matplotlib.rcParams["axes.xmargin"] = 0

# Normal force feedback sweep

## Sweep variable init

In [ ]:
N_constant_ff_Fs = [1, 3, 5]
theta_Ls = []

## Other init

In [ ]:
ctrl_paradigm = plant.simulation.CtrlParadigm.IMPEDANCE
impedance_type = plant.simulation.ImpedanceType.LINK_FB
n_hat_force_compensation_source = plant.simulation.NHatForceCompensationSource.CONSTANT

## Run sweep

In [ ]:
for N_constant_ff_F in N_constant_ff_Fs:
    # Set up sim
    sim = plant.simulation.Simulation(
        ctrl_paradigm=ctrl_paradigm,
        impedance_type=impedance_type,
        n_hat_force_compensation_source=n_hat_force_compensation_source,
        exit_when_folded=True,
        N_constant_ff_F=N_constant_ff_F
    )
    print(sim.N_constant_ff_F)

    # Run sim
    t_start_ = time.time()
    log = sim.run_sim()
    print(time.time() - t_start_)
    
    # Grab output var
    theta_L = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx) + hinge_rotation_axis].copy()
    theta_LZ = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx)+2]
    # Fix issue in RPY singularity
    theta_L[theta_LZ > np.pi/2] = theta_L[theta_LZ > np.pi/2]*-1 + np.pi
    
    theta_Ls.append(np.max(theta_L))

## Plot

In [ ]:
plt.plot(N_constant_ff_Fs, theta_Ls)
plt.show()

# Impedance sweep

## Sweep variable init

In [ ]:
base_impedance = np.array([1, 1, 1, 10, 10, 10])
impedance_scales = [0.1, 1, 10]

In [ ]:
theta_Ls = []

## Other init

In [ ]:
ctrl_paradigm = plant.simulation.CtrlParadigm.IMPEDANCE
impedance_type = plant.simulation.ImpedanceType.LINK_FB
n_hat_force_compensation_source = plant.simulation.NHatForceCompensationSource.CONSTANT

## Run sweep

In [ ]:
for imp in impedance_scales:
    # Set up sim
    sim = plant.simulation.Simulation(
        ctrl_paradigm=ctrl_paradigm,
        impedance_type=impedance_type,
        n_hat_force_compensation_source=n_hat_force_compensation_source,
        exit_when_folded=True,
        N_constant_ff_F=5,
        impedance_stiffness=imp*base_impedance
    )
    print(imp*base_impedance)

    # Run sim
    t_start_ = time.time()
    log = sim.run_sim()
    print(time.time() - t_start_)
    
    # Grab output var
    theta_L = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx) + hinge_rotation_axis].copy()
    theta_LZ = log.data()[sim.log_wrapper.get_idx("pos", "rot", sim.ll_idx)+2]
    # Fix issue in RPY singularity
    theta_L[theta_LZ > np.pi/2] = theta_L[theta_LZ > np.pi/2]*-1 + np.pi
    plt.figure()
    plt.plot(log.sample_times(), theta_L)
    plt.show()
    
    theta_Ls.append(np.max(theta_L))

## Plot

In [ ]:
plt.plot(impedance_scales, theta_Ls, '--o', linewidth=2)
plt.show()